# Analyse et Modélisation des Prix de Véhicules
Créer un modèle pour prédire le prix d’un véhicule à partir de caractéristiques techniques et visuelles, avec un dataset sale, non structuré et plein de chaînes de caractères mal formatées.

🔗 Dataset :
👉 https://www.kaggle.com/datasets/CooperUnion/cardataset

## 1. Compréhension et audit du dataset
- Charger les données
- Identifier les types de variables (numériques, catégorielles, chaînes)
- Lister les incohérences et valeurs manquantes
- Questions : colonnes utiles, corrélations potentielles, doublons/rédundances



In [ ]:
import pandas as pd
import numpy as np
# Charger le dataset
# df = pd.read_csv('path_to_data.csv')

## 2. Nettoyage avancé
- Homogénéiser le format des prix
- Normaliser la cylindrée et la puissance
- Corriger les formats de date
- Uniformiser marques et modèles
- Questions : unités de puissance, traitement des lignes incohérentes


Exemples de nettoyage :
Prix : convertir "20,000 USD", "23k", "€19.5K" → nombre homogène

Cylindrée : "2.0L", "2000cc", "120 hp" → puissance normalisée

Dates : corriger les années absurdes (ex: fabrication après la vente)

Marques et modèles : uniformiser les noms ("VW", "Volkswagen", "volks wagen")

In [ ]:
# Exemple de nettoyage des prix
# df['price_clean'] = df['price'].apply(clean_price)

## 3. Feature Engineering (création de variables)
- Calcul de l'âge du véhicule
- Catégorisation du segment (compact, SUV, etc.)
- Encodage des variables catégorielles
- Questions : relation âge/prix, influence du segment


Exemples :
Age du véhicule = année actuelle - année de fabrication

Segment du véhicule (ex: compact, SUV, utilitaire) à partir du modèle

Encodeur ordinal ou one-hot pour les catégories (carburant, boîte auto/manuelle, etc.)

In [ ]:
# Création de nouvelles variables
# df['age'] = current_year - df['year']

## 4. Analyse exploratoire (EDA)
- Corrélations entre variables numériques
- Distribution du prix par segment et type de carburant
- Identification des outliers sur le prix et le kilométrage
- Analyse de la distribution des variables (histogrammes, boxplots)
- Analyse des interactions entre variables (scatter matrix)

**Questions à se poser** :
- Quelles variables numériques sont les plus corrélées avec le prix ?
- Le prix suit-il une relation linéaire ou log-linéaire avec l'âge, le kilométrage et la puissance ?
- Y a-t-il des effets de seuil ou de segments (par exemple SUV vs citadine) sur la distribution des prix ?
- Comment le kilométrage impacte-t-il le prix selon l'âge du véhicule ?
- Y a-t-il des relations non linéaires entre la puissance moteur et le prix ?
- La localisation géographique ou la date d'inscription influence-t-elle significativement le prix ?
- Quels attributs catégoriels (carburant, transmission, couleur, nombre de portes) ont un impact sur le prix ?
- Y a-t-il des valeurs aberrantes dans d'autres variables (kilométrage, âge) qui pourraient fausser les analyses ?
- Les variables formant des paires montrent-elles des interactions (par exemple âge vs puissance) ?
- Faut-il transformer certaines variables (log, racine carrée) pour linéariser leur relation avec le prix ?

In [ ]:
# Visualisations
# import matplotlib.pyplot as plt

## 5. Modélisation (sans scikit-learn)
- Régression linéaire avec NumPy / statsmodels
- Évaluation performance sur validation
- Questions : variables importantes, sur-/sous-apprentissage

In [ ]:
# Modèle de régression
# import statsmodels.api as sm

## 6. Analyse des erreurs et des résidus
- Distribution des erreurs par segment
- Résidus vs prix réel
- Détection d'outliers non captés


In [ ]:
# Analyse des résidus
# residuals = model.predict(X) - y_true

## 7. Synthèse des résultats
- Importance des variables
- Exemples de bonnes/mauvaises prédictions
- Limites et pistes d'amélioration
